import libraries, assign column names for pandas, search url for specific class and capute that data

In [168]:
# import libraries
import json, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# assign column names
columns = ['PostalCode', 'Borough', 'Neighborhood']

# get html from url using lxml parser and return only data within the wikitable sortable class
resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(resp, 'lxml')
data = soup.find('table',{'class':'wikitable sortable'})

iterate through captured data and place each row into array as an array

In [169]:
# iterate through rows and save as multideminsional array
sites = []
for row in data.findAll('tr')[1:]:
    cells = row.find_all(['td', 'th'])
   
    try:
        postalCode, borough, neighborhood = cells[0].text, cells[1].text, cells[2].text
    except IndexError:
        continue

    sites.append([postalCode, borough, neighborhood.rstrip()])

create dataFrame, remove Not assigned Borough and reassign Not assigned Neighborhood with Borough

In [170]:
# create dataFrame from data and assign column names
df = pd.DataFrame(sites, columns=columns)

# drop Not assigned rows for Borough
df = df[~df['Borough'].isin(['Not assigned'])]
# df = df[~df['Neighborhood'].isin(['Not assigned'])]

# assign Borough to Neighborhood that has Not assigned
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']


groupby PostalCode and Borough and apply join on Neighborhood. convert series to dataFrame and reset index.

In [171]:
# groupby PostalCode and apply join to Neighborhood. Convert series to dataFrame and reset index
df = df.groupby(['PostalCode', 'Borough'], sort=False)['Neighborhood'].apply(', '.join).to_frame().reset_index(drop=False)

print(df.shape)

(103, 3)
